In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import pickle
routes = pickle.load( open( "save.p", "rb" ) )
routes

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,polyline array
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[[41.154489, -8.630838], [41.154498, -8.630829..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...","[[41.170671, -8.66574], [41.170635, -8.665767]..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...","[[41.14053, -8.61597], [41.140863, -8.617365],..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...","[[41.142915, -8.607996], [41.142915, -8.608005..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...","[[41.178087, -8.687268], [41.178078, -8.687259..."
...,...,...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,A,False,"[[-8.612469,41.14602],[-8.612487,41.145993],[-...","[[41.171013, -8.611344], [41.170995, -8.611353..."
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,A,False,"[[-8.610138,41.140845],[-8.610174,41.140935],[...","[[41.158755, -8.627454], [41.158746, -8.627427..."
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,A,False,[],[]
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,A,False,"[[-8.630712,41.154885],[-8.63073,41.154813],[-...","[[41.173524, -8.587026], [41.172138, -8.587791..."


In [3]:
routes['duration'] = (routes['polyline array'].str.len()-1)*15/60

In [4]:
sample_route = routes['polyline array'].loc[0]
#sample_route

In [32]:
#np.flip(sample_route)

In [33]:
#plt.plot(*zip(*sample_route))

## Mapping

In [34]:
# %%time
# plt.figure(figsize=(30,20))

# for route in routes['polyline array'].iloc[:1000]:
#     plt.plot(*zip(*route))

In [5]:
import os 
import folium
from folium import plugins
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
#import earthpy as et

In [6]:
m = folium.Map(location=[41.1579, -8.6291],
              tiles = 'Stamen Terrain')

In [7]:
%%time
for route in routes['polyline array'].iloc[:50]:
    folium.PolyLine(route, color="magenta", weight=1.5, opacity=1).add_to(m)

Wall time: 14.7 ms


In [64]:
m

## Classification

In [11]:
#convert timestamps
from datetime import datetime as dt

In [24]:
routes['dt_start'] = routes['TIMESTAMP'].apply(lambda x: dt.fromtimestamp(x))

In [19]:
#pull out info
routes['hour'] = routes['dt_start'].dt.hour
routes['day'] = routes['dt_start'].apply(lambda x: dt.weekday(x)) #monday = 0, sunday = 6

In [58]:
##Work in Progress
#pull out origin....this works for v small set, but not working for whole set..
orgs = []
k = routes[:5]
for i in range(len(k)):    
        x = routes['polyline array'].loc[i][0]
        orgs.append(x)
len(orgs)

5

In [23]:
routes.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,polyline array,duration,dt_start,hour,day
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[[41.154489, -8.630838], [41.154498, -8.630829...",5.5,2013-06-30 19:00:58,19,6
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...","[[41.170671, -8.66574], [41.170635, -8.665767]...",4.5,2013-06-30 19:08:23,19,6
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...","[[41.14053, -8.61597], [41.140863, -8.617365],...",16.0,2013-06-30 19:02:31,19,6
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...","[[41.142915, -8.607996], [41.142915, -8.608005...",10.5,2013-06-30 19:00:54,19,6
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...","[[41.178087, -8.687268], [41.178078, -8.687259...",7.0,2013-06-30 19:04:51,19,6
